<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/FE_working_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch1.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch2.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
pip install medpy

In [ ]:
from medpy.io import load, save
import os
import os.path
import gc
from scipy.ndimage import zoom
import numpy as np
from tqdm import tqdm
try:
  os.mkdir('data')
except:
  pass
for file in tqdm(range(28)):
        shape=64/2
        img, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-'+str(file)+'.nii')
        seg, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-'+str(file)+'.nii')
        arr=np.where(seg==2)[-1]
        mn=(arr.min()+arr.max())/2
        
        
        if mn-shape<0:
          start=0
          end=int(shape*2)
        elif mn+shape>img.shape[-1]:
          end=int(img.shape[-1])
          start=int(end-shape*2)
        else:
          start=int(max(0,(mn-shape)))
          end=int(min(img.shape[-1],int(mn+shape)))
        
        
        img=img[:,:,start:end]
        img[img < -250] = -250
        img[img > 250] = 250
        img=(img+250)/500
        img = np.array(img, dtype='float32')
        # img = zoom(img, (0.125, 0.125, 1))
        # print ("Saving image "+str(file))
        # np.save( "/content/data/volume-" + str(file) ,zoom(img, (0.125, 0.125, 1)))
        # print(img.shape)
        # del([img])
        # gc.collect()
        seg=seg.astype('float64')
        seg/=2
        # seg+=0.1645
        # seg/=0.023138
        seg = np.array(seg, dtype='float32')
        seg=seg[:,:,start:end]
        # print ("Saving image "+file)
        # seg=zoom(seg, (0.125, 0.125, 1))
        total=np.concatenate([img,seg],-1)
        print(total.shape)
        np.save( "/content/data/total-" + str(file),total)
        del([seg,img,total])
        gc.collect()

In [ ]:
from medpy.io import load, save
import os
import os.path
import gc
from scipy.ndimage import zoom
import numpy as np
from tqdm import tqdm
try:
  os.mkdir('data')
except:
  pass
for file in tqdm(range(28,131)):
        shape=64/2
        img, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 2/volume-'+str(file)+'.nii')
        seg, img_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 2/segmentation-'+str(file)+'.nii')
        
        arr=np.where(seg==2)[-1]
        if len(arr)==0:
            arr=np.asarray([0,img.shape[-1]])
        
        mn=(arr.min()+arr.max())/2
        
        
        if mn-shape<0:
          start=0
          end=int(shape*2)
        elif mn+shape>img.shape[-1]:
          end=int(img.shape[-1])
          start=int(end-shape*2)
        else:
          start=int(max(0,(mn-shape)))
          end=int(min(img.shape[-1],int(mn+shape)))
        
        
        img=img[:,:,start:end].astype('float64')
        img[img < -250] = -250
        img[img > 250] = 250
        img=(img+250)/500
        img = np.array(img, dtype='float32')
        # img = zoom(img, (0.125, 0.125, 1))
        # print ("Saving image "+str(file))
        # np.save( "/content/data/volume-" + str(file) ,zoom(img, (0.125, 0.125, 1)))
        # print(img.shape)
        # del([img])
        # gc.collect()
        seg=seg.astype('float64')
        seg/=2

        # seg+=0.1645
        # seg/=0.023138
        seg = np.array(seg, dtype='float32')
        seg=seg[:,:,start:end]
        # print ("Saving image "+file)
        # seg=zoom(seg, (0.125, 0.125, 1))
        total=np.concatenate([img,seg],-1)
        print(total.shape)
        np.save( "/content/data/total-" + str(file),total)
        del([seg,img,total])
        gc.collect()

In [ ]:
pip install nilearn

In [ ]:
%pylab inline
import numpy as np
import torch
import os

from torch import nn
from torch import optim
from torch.nn import functional as F
from torch import autograd
from torch.autograd import Variable
import nibabel as nib
from torch.utils.data.dataset import Dataset
from torch.utils.data import dataloader
from nilearn import plotting


BATCH_SIZE=2
max_epoch = 100
lr = 0.0001
gpu = True
workers = 4

LAMBDA= 10
#setting latent variable sizes
latent_dim = 1000

import csv
import numpy as np
import torch
from torch.utils.data.dataset import Dataset
import os
from torchvision import transforms
from skimage.transform import resize
from nilearn import surface
from medpy.io import *
import glob
class ADNIdataset(Dataset):
  def __init__(self,range, augmentation=False):
    self.augmentation = augmentation
    self.name = []
    self.range=range
    for data in self.range:
    	self.name.append('/content/data/total-'+str(data)+'.npy')
    self.name = np.asarray(self.name)

  def __len__(self):
  	return len(self.name)

  def __getitem__(self, index):
  	path = self.name[index]
  	img = np.load(path)

  	img = np.flip(img,1)
  	sp_size = 64
  	img = resize(img, (sp_size,sp_size,sp_size), mode='constant')
    img = (img-np.min(img))/(np.max(img)-np.min(img))
  	
    if self.augmentation:
      random_n = torch.rand(1)
      if random_n[0] > 0.5:
        img = np.flip(img,0)
    
    img = np.ascontiguousarray(img,dtype=np.float32)
  	imageout = torch.from_numpy(img).float().view(1,sp_size,sp_size,sp_size)
  	imageout = imageout*2-1

  	return imageout

def calc_gradient_penalty(netD, real_data, fake_data):    
    alpha = torch.rand(real_data.size(0),1,1,1,1)
    alpha = alpha.expand(real_data.size())
    
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.cuda()
    interpolates = Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty
def inf_train_gen(data_loader):
    while True:
        for _,images in enumerate(data_loader):
            yield images

In [ ]:

import numpy as np
import torch
import os
from torch import nn
from torch import optim
from torch.nn import functional as F

class Discriminator(nn.Module):
    def __init__(self, channel=512):
        super(Discriminator, self).__init__()        
        self.channel = channel
        n_class = 1
        
        self.conv1 = nn.Conv3d(1, channel//8, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv3d(channel//8, channel//4, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm3d(channel//4)
        self.conv3 = nn.Conv3d(channel//4, channel//2, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm3d(channel//2)
        self.conv4 = nn.Conv3d(channel//2, channel, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm3d(channel)
        
        self.conv5 = nn.Conv3d(channel, n_class, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, _return_activations=False):
        h1 = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        h2 = F.leaky_relu(self.bn2(self.conv2(h1)), negative_slope=0.2)
        h3 = F.leaky_relu(self.bn3(self.conv3(h2)), negative_slope=0.2)
        h4 = F.leaky_relu(self.bn4(self.conv4(h3)), negative_slope=0.2)
        h5 = self.conv5(h4)
        output = h5

        return output
    

class Generator(nn.Module):
    def __init__(self, noise:int=1000, channel:int=64):
        super(Generator, self).__init__()
        _c = channel
        
        self.noise = noise
        self.fc = nn.Linear(1000,512*4*4*4)
        self.fc.requires_grad_=False
        self.bn1 = nn.BatchNorm3d(_c*8)
        self.bn1.requires_grad_=False
        
        self.tp_conv2 = nn.Conv3d(_c*8, _c*4, kernel_size=3, stride=1, padding=1, bias=False)
        self.tp_conv2.requires_grad_=False
        self.bn2 = nn.BatchNorm3d(_c*4)
        self.bn2.requires_grad_=False
        
        self.tp_conv3 = nn.Conv3d(_c*4, _c*2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm3d(_c*2)
        
        self.tp_conv4 = nn.Conv3d(_c*2, _c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm3d(_c)
        
        self.tp_conv5 = nn.Conv3d(_c, 1, kernel_size=3, stride=1, padding=1, bias=False)
        
    def forward(self, noise):
        noise = noise.view(-1, 1000)
        h = self.fc(noise)
        h = h.view(-1,512,4,4,4)
        h = F.relu(self.bn1(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv2(h)
        h = F.relu(self.bn2(h))
        
        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv3(h)
        h = F.relu(self.bn3(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv4(h)
        h = F.relu(self.bn4(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv5(h)

        h = F.tanh(h)

        return h

In [ ]:
trainset1 = ADNIdataset(list(range(65)),augmentation=True)
train_loader1 = torch.utils.data.DataLoader(trainset1,batch_size=BATCH_SIZE,
                                          shuffle=True,num_workers=workers)
trainset2 = ADNIdataset(list(range(65,131)),augmentation=True)
train_loader2 = torch.utils.data.DataLoader(trainset2,batch_size=BATCH_SIZE,
                                          shuffle=True,num_workers=workers)



D1 = Discriminator().cuda()
G1 = Generator().cuda()
a=torch.load('/content/gdrive/MyDrive/segmentation/WGAN_G(1).pth',map_location='cuda:0')
G1.load_state_dict(a)
g_optimizer1 = optim.Adam(G1.parameters(), lr=0.0002)
d_optimizer1 = optim.Adam(D1.parameters(), lr=0.0002)


D2 = Discriminator().cuda()
G2 = Generator().cuda()
G2.load_state_dict(a)
g_optimizer2 = optim.Adam(G2.parameters(), lr=0.0002)
d_optimizer2 = optim.Adam(D2.parameters(), lr=0.0002)


D_org = Discriminator().cuda()
G_org = Generator().cuda()

real_y = Variable(torch.ones((BATCH_SIZE, 1)).cuda())
fake_y = Variable(torch.zeros((BATCH_SIZE, 1)).cuda())
loss_f = nn.BCELoss()




In [ ]:
Ds=[D1,D2]
Gs=[G1,G2]
train_loaders=[inf_train_gen(train_loader1),inf_train_gen(train_loader2)]
d_optimizers=[d_optimizer1,d_optimizer2]
g_optimizers=[g_optimizer1,g_optimizer2]

In [ ]:
from tqdm import tqdm
import torch.distributions as tdist
TOTAL_ITER = 66
pace=10
for epoch in range(200):
 for iteration in tqdm(range(TOTAL_ITER)):
  for num in range(2):
    ###############################################
    # Train D 
    ###############################################
    d_optimizer=d_optimizers[num]
    g_optimizer=g_optimizers[num]
    D=Ds[num]
    G=Gs[num]
    for p in D.parameters():  
        p.requires_grad = True 

    real_images = train_loaders[num].__next__()
    if real_images.shape[0]==1:
      real_images = train_loaders[num].__next__()
      
    D.zero_grad()
    real_images = Variable(real_images).cuda()

    _batch_size = real_images.size(0)


    y_real_pred = D(real_images)

    d_real_loss = y_real_pred.mean()
    
    noise = Variable(torch.randn((_batch_size, latent_dim, 1, 1, 1)),volatile=True).cuda()
    fake_images = G(noise)
    y_fake_pred = D(fake_images.detach())

    d_fake_loss = y_fake_pred.mean()

    gradient_penalty = calc_gradient_penalty(D,real_images.data, fake_images.data)
 
    d_loss = - d_real_loss + d_fake_loss +gradient_penalty
    d_loss.backward()
    Wasserstein_D = d_real_loss - d_fake_loss

    d_optimizer.step()

    ###############################################
    # Train G 
    ###############################################
    for p in D.parameters():
        p.requires_grad = False
        
    for iters in range(5):
        G.zero_grad()
        noise = Variable(torch.randn((_batch_size, latent_dim, 1, 1 ,1)).cuda())
        fake_image =G(noise)
        y_fake_g = D(fake_image)

        g_loss = -y_fake_g.mean()

        g_loss.backward()
        g_optimizer.step()
  device='cuda'
  if iteration%pace ==0:
    with torch.no_grad():
        for key in D_org.state_dict().keys():
            
            if Ds[0].state_dict()[key].dtype == torch.int64:
                D_org.state_dict()[key].data.copy_(Ds[0].state_dict()[key])
            else:
                temp = torch.zeros_like(D_org.state_dict()[key])
                for s in range(2):
                    try:
                      nn = tdist.Normal(torch.tensor([0.0]), 1e-5*torch.std(Ds[s].state_dict()[key].detach().cpu()))
                    except:
                      nn = tdist.Normal(torch.tensor([0.0]), 1e-5*1e-3)
                    shape=Ds[s].state_dict()[key].shape
                    noise = nn.sample(shape).reshape(shape)
                    noise = noise.to(device)
                    if s==0:
                      t_noise=noise
                    else:
                      t_noise+=noise
                    temp += 0.5*(Ds[s].state_dict()[key]+noise)
                D_org.state_dict()[key].data.copy_(temp)
                
                
                for s in range(2):
                        Ds[s].state_dict()[key].data.copy_(D_org.state_dict()[key])



        for key in G_org.state_dict().keys():
            if Gs[0].state_dict()[key].dtype == torch.int64:
                G_org.state_dict()[key].data.copy_(Gs[0].state_dict()[key])
            else:
                temp = torch.zeros_like(G_org.state_dict()[key])
                for s in range(2):
                    try:
                      nn = tdist.Normal(torch.tensor([0.0]), 1e-5*torch.std(Gs[s].state_dict()[key].detach().cpu()))
                    except:
                      nn = tdist.Normal(torch.tensor([0.0]), 1e-5*1e-3)
                    shape=Gs[s].state_dict()[key].shape
                    noise = nn.sample(shape).reshape(shape)
                    noise = noise.to(device)
                    if s==0:
                      t_noise=noise
                    else:
                      t_noise+=noise
                    temp += 0.5*(Gs[s].state_dict()[key]+noise)
                G_org.state_dict()[key].data.copy_(temp)
                for s in range(2):
                        Gs[s].state_dict()[key].data.copy_(G_org.state_dict()[key])
 torch.save({
            'model_state_dict': G_org.state_dict(),
            'optimizer_state_dict1': g_optimizer1.state_dict(),
            'optimizer_state_dict2': g_optimizer2.state_dict()
            }, '/content/gdrive/MyDrive/generator_'+str(epoch)+'.pth')
 torch.save({
            'model_state_dict': D_org.state_dict(),
            'optimizer_state_dict1': d_optimizer1.state_dict(),
            'optimizer_state_dict2': d_optimizer2.state_dict()
            }, '/content/gdrive/MyDrive/discriminator_'+str(epoch)+'.pth')

Streaming output truncated to the last 5000 lines.

 41%|████      | 27/66 [02:24<03:56,  6.06s/it]



 42%|████▏     | 28/66 [02:28<03:23,  5.36s/it]



 44%|████▍     | 29/66 [02:32<03:00,  4.87s/it]



 45%|████▌     | 30/66 [02:35<02:42,  4.52s/it]



 47%|████▋     | 31/66 [02:47<03:54,  6.69s/it]



 48%|████▊     | 32/66 [02:51<03:14,  5.72s/it]



 50%|█████     | 33/66 [02:54<02:49,  5.12s/it]



 52%|█████▏    | 34/66 [03:09<04:12,  7.89s/it]



 53%|█████▎    | 35/66 [03:12<03:25,  6.64s/it]



 55%|█████▍    | 36/66 [03:16<02:52,  5.76s/it]



 56%|█████▌    | 37/66 [03:20<02:29,  5.15s/it]



 58%|█████▊    | 38/66 [03:24<02:12,  4.72s/it]



 59%|█████▉    | 39/66 [03:27<01:59,  4.43s/it]



 61%|██████    | 40/66 [03:31<01:49,  4.22s/it]



 62%|██████▏   | 41/66 [03:36<01:54,  4.57s/it]



 64%|██████▎   | 42/66 [03:40<01:42,  4.25s/it]



 65%|██████▌   | 43/66 [03:44<01:34,  4.10s/it]



 67%|██████▋   | 44/66 [03:47<01:27,  3.98s/it]



 68%|██████▊   | 45/66 [03:51<